# **Decision Trees**

The Wisconsin Breast Cancer Dataset(WBCD) can be found here(https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data)

This dataset describes the characteristics of the cell nuclei of various patients with and without breast cancer. The task is to classify a decision tree to predict if a patient has a benign or a malignant tumour based on these features.

Attribute Information:
```
#  Attribute                     Domain
   -- -----------------------------------------
   1. Sample code number            id number
   2. Clump Thickness               1 - 10
   3. Uniformity of Cell Size       1 - 10
   4. Uniformity of Cell Shape      1 - 10
   5. Marginal Adhesion             1 - 10
   6. Single Epithelial Cell Size   1 - 10
   7. Bare Nuclei                   1 - 10
   8. Bland Chromatin               1 - 10
   9. Normal Nucleoli               1 - 10
  10. Mitoses                       1 - 10
  11. Class:                        (2 for benign, 4 for malignant)
```

In [ ]:
import numpy as np 
import pandas as pd 
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import train_test_split 
from sklearn import tree

In [ ]:
headers = ["ID","CT","UCSize","UCShape","MA","SECSize","BN","BC","NN","Mitoses","Diagnosis"]
data = pd.read_csv('breast-cancer-wisconsin.data', na_values='?',    
         header=None, index_col=['ID'], names = headers) 
data = data.reset_index(drop=True)
data = data.fillna(0)
data

,CT,UCSize,UCShape,MA,SECSize,BN,BC,NN,Mitoses,Diagnosis
0,5,1,1,1,2,1.0,3,1,1,2
1,5,4,4,5,7,10.0,3,2,1,2
2,3,1,1,1,2,2.0,3,1,1,2
3,6,8,8,1,3,4.0,3,7,1,2
4,4,1,1,3,2,1.0,3,1,1,2
...,...,...,...,...,...,...,...,...,...,...
694,3,1,1,1,3,2.0,1,1,1,2
695,2,1,1,1,2,1.0,1,1,1,2
696,5,10,10,3,7,3.0,8,10,2,4
697,4,8,6,4,3,4.0,10,6,1,4


1. a) Implement a decision tree (you can use decision tree implementation from existing libraries).

In [ ]:
# train test split
X = data.values[:, :-1] 
Y = data.values[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 12) 

1. b) Train a decision tree object of the above class on the WBC dataset using misclassification rate, entropy and Gini as the splitting metrics.

In [ ]:
# gini
clf_gini = DecisionTreeClassifier(criterion = "gini", random_state = 100,max_depth=5)
clf_gini.fit(X_train, y_train)
y_gini_pred = clf_gini.predict(X_test)

In [ ]:
# entropy
clf_entropy = DecisionTreeClassifier(criterion = "entropy", random_state = 100,max_depth=5)
clf_entropy.fit(X_train, y_train)
y_entropy_pred = clf_entropy.predict(X_test)

1. c) Report the accuracies in each of the above splitting metrics and give the best result. 

In [ ]:
print("Accuracies:")
print('Gini:',accuracy_score(y_test,y_gini_pred)*100,'%')
print('Entropy:',accuracy_score(y_test,y_entropy_pred)*100,'%')

Accuracies:
Gini: 95.0 %
Entropy: 96.42857142857143 %


1. d) Experiment with different approaches to decide when to terminate the tree (number of layers, purity measure, etc). Report and give explanations for all approaches. 

- We used RandomizedCV and GridSearchCV (from scikit-learn) for model selection, we get the best possible model for decision trees. Most of the models are giving nearly 95%-96% accuracy. 
- Manual model selection was also done, it gave similar results. But these approaches seemed exhaustive and better for hyper-parameter tuning. For max_depth we tried to keep it low so as to avoid overfitting.

- GridSearchCV
  - A very common approach for hyperparameter tuning in scikit-learn is GridSearchCV. GridSearchCV is an exhaustive approach, because it checks every single possible combination of hyperparameter values to determine the optimal combination.
- RandomizedSearchCV
  - This method uses clever shortcut — rather than trying every single unique combination of hyperparameter values, a random search samples a randomly-selected subset of n combinations. The randomized search process requires considerably less compute time and often delivers a similar result. The logic is that by checking enough randomly-chosen combinations on the grid, the search is likely to identify one that is similar to that of an exhaustive process would have identified.

In [ ]:
# hyper-parameter tuning
from sklearn.model_selection import RandomizedSearchCV

hyper_params = {
    'criterion': ['gini','entropy'],
    'max_depth': range(3,20),
    'min_samples_leaf': range(5,400),
    'random_state': range(0,100)
}
clf = RandomizedSearchCV(DecisionTreeClassifier(), hyper_params, cv=5)

In [ ]:
clf.fit(X_train,y_train)
print(clf.best_params_,'Score:',clf.best_score_)

{'random_state': 97, 'min_samples_leaf': 6, 'max_depth': 4, 'criterion': 'gini'} Score: 0.9409427284427284


In [ ]:
# best parameter using RandomizedSearchCV
clf_entropy = DecisionTreeClassifier(criterion = "entropy", random_state = 59, max_depth=5)
clf_entropy.fit(X_train, y_train)
y_entropy_pred = clf_entropy.predict(X_test)
print('Entropy:',accuracy_score(y_test,y_entropy_pred)*100,'%')

Entropy: 96.42857142857143 %


In [ ]:
from sklearn.model_selection import GridSearchCV

hyper_params = {
    'criterion': ['gini','entropy'],
    'max_depth': range(3,20),
    'min_samples_leaf': range(5,100,10),
    'random_state': range(0,100,25)
}
clf = GridSearchCV(DecisionTreeClassifier(), hyper_params, cv=5)
clf.fit(X_train,y_train)
print(clf.best_params_,'Score:',clf.best_score_)

{'criterion': 'entropy', 'max_depth': 3, 'min_samples_leaf': 15, 'random_state': 0} Score: 0.9535231660231659


In [ ]:
# best parameter using GridSearchCV
clf_entropy = DecisionTreeClassifier(criterion = "entropy", random_state = 0, max_depth=3)
clf_entropy.fit(X_train, y_train)
y_entropy_pred = clf_entropy.predict(X_test)
print('Entropy:',accuracy_score(y_test,y_entropy_pred)*100,'%')

Entropy: 95.0 %


2. What is boosting, bagging and  stacking?
Which class does random forests belong to and why?

Answer:    
Boosting, bagging and  stacking are very popular ensemble techniques.
- **Boosting**:
  (Weighted voting)    
The idea of boosting is to train weak learners sequentially, each trying to correct its predecessor. The weak learners are sequentially corrected by their predecessors and, in the process, they are converted into strong learners. While combining , boosting pays higher focus on examples which are miss-classified or have higher errors by preceding weak rules.

- **Bagging**:   
  Objective to reduce variance, not bias of a decision tree. It creates a few subsets of data from the training sample, which is chosen randomly with replacement. Now each collection of subset data is used to prepare their decision trees thus, we end up with an ensemble of various models. The average of all the assumptions from numerous tress is used, which is more powerful than a single decision tree.

- **Stacking**:   
  Stacking learns from some weak models but as opposed to above two methods the weak learners here are heterogenous (i.e, different learning algorithms are used) and also as opposed to deterministic combining strategy used in above two methods, it learns to combine the base models.

- Random forest belongs to bagging class because random forest is an averaging method that aims to reduce the variance of individual trees by randomly selecting many trees from the dataset, and averaging them. So we see that all the weak learners here are decision trees that are trained on a subset of trainng data and while combining mode of the observation is taken, therefore all the decisoin trees are uncorelated. Therefore it belongs to bagging approach.

3. Implement random forest algorithm using different decision trees . 

In [ ]:
# default values are chosen from documentation: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
# min_size = min_samples_split as in docs
# n_features = subset of features in a decision tree
class RandomForest:
  def __init__(self, depth=None, min_size=2, n_trees=100, criterion='gini'):
    self.X_train, self.y_train, self.depth, self.min_size, self.n_trees, self.criterion = X_train, y_train, depth, min_size, n_trees, criterion
    self.dTrees = [self.createTree() for i in range(n_trees)]
  
  def createTree(self):
    return DecisionTreeClassifier(max_depth=self.depth, min_samples_split=self.min_size, criterion=self.criterion)

  # n_features is in fit because we should be able to change it according to the data (X_train)
  def fit(self, X_train, y_train, n_features=None):
    np.random.seed(10)
    if n_features == None:
      self.n_features = int(np.sqrt(X_train.shape[1]))
    else:
      self.n_features = n_features

    self.indexes = []
    for tree_clf in self.dTrees:
      # idx = np.random.randint(0,X_train.shape[1]-1,self.n_features)
      idx = np.random.permutation(X_train.shape[1])[:self.n_features]
      self.indexes.append(idx)

      # print(self.indexes)
      X_new = []
      for i in idx:
        X_new.append(X_train[:,i])
      X_new = np.array(X_new).T
      y_new = np.array(y_train)
      
      # decision tree fitting (not RandomForest fitting)
      tree_clf.fit(X_new,y_new)

  def predict(self, X_test):
    prediction_list = []
    for i in range(self.n_trees):
      idx = self.indexes[i]
      X_test_new = []
      for i in idx:
        X_test_new.append(X_test[:,i])
      X_test_new = np.array(X_test_new).T 
      # print(X_test_new.shape)
      y_pred = self.dTrees[i].predict(X_test_new)
      # y_pred = np.array(y_pred)
      # print(y_pred.shape)

      prediction_list.append(y_pred)
    # prediction_list = np.array(prediction_list)
    # print(prediction_list.shape)  # (100=n_trees,140=rows)
    prediction_list = np.array(prediction_list).T
    prediction_list = prediction_list.astype(int)

    voted_y_pred = []
    for row in prediction_list:
      voted_y_pred.append(np.argmax(np.bincount(row)))
    # print(voted_y_pred)
    return voted_y_pred

4. Report the accuracies obtained after using the Random forest algorithm and compare it with the best accuracies obtained with the decision trees. 

In [ ]:
model = RandomForest(depth=3,n_trees=10,criterion='entropy')
model.fit(X_train,y_train, 4)
y_pred = model.predict(X_test)

In [ ]:
print('Accuracy:',accuracy_score(y_test,y_pred)*100,'%')

Accuracy: 97.85714285714285 %


The best accuracies of the decision trees reached till 96%, but random forest performs significantly better at almost 98% accuracy.

5. Submit your solution as a separate pdf in the final zip file of your submission


Compute a decision tree with the goal to predict the food review based on its smell, taste and portion size.

(a) Compute the entropy of each rule in the first stage.

(b) Show the final decision tree. Clearly draw it.

Submit a handwritten response. Clearly show all the steps.

>**Refer dtree.pdf**